In [26]:
from src.downloader import *
from src.detector import Detector
import numpy as np

In [27]:
# fpath = download('https://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20210904112822A5.pdf')
images = convert_image('20220209135647A8.pdf')
images.shape

(15, 2526, 1797, 3)

In [28]:
detector = Detector(images)

In [29]:
out = detector.detect()
out['line'].shape,out['imaeg_id'].shape

((525, 4), (525,))

In [30]:
image = images[3]
text_bbox = detector.detect_text(image)
text_bbox = text_bbox[detector.filter(text_bbox)]

In [31]:
text_bbox[:, 2:4].mean(axis=0).astype(np.int8)

array([124,  31], dtype=int8)

In [32]:
from PIL import Image
import cv2

def calculate_ratio(width,height):
    ratio = width/height
    if ratio<1.0:
        ratio = 1./ratio
    return ratio
  
def compute_ratio_and_resize(img,width,height,model_height):
    ratio = width/height
    if ratio<1.0:
        ratio = 1./ratio
        new_size = (model_height,int(model_height*ratio))
    else:
        new_size = (int(model_height*ratio),model_height)
    img = cv2.resize(img, new_size,interpolation=Image.ANTIALIAS)
    return img, ratio

In [33]:
def get_image_list(image, bboxes):
    sub_images = list()

    for x, y, w, h in bboxes:
        im = image[y:y+h, x:x+w]
        # trans_im, _ = compute_ratio_and_resize(im, w, h, 64)
        sub_images.append(im)
    
    # sub_iamge_arr = np.array(sub_images)
    return sub_images

In [34]:
import easyocr
import torch
from easyocr.recognition import AlignCollate

In [47]:
imdim = (64,((text_bbox[:,2]/text_bbox[:,3]).mean().astype(int)*64))
image_list = get_image_list(image, text_bbox)

In [57]:
reader = easyocr.Reader(
  ['th'],
  recog_network='thai-vl',
  model_storage_directory='src/model',
  user_network_directory='src/model',
  detect_network='dbnet18'
  )

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Compiling deform_conv_cpu ...
(This may take a while if this module is loaded for the first time.)


In [55]:
reader.readtext(images[0])

KeyboardInterrupt: 